Connected to Python 3.12.2

In [ ]:
import torch
import sys
import os
import numpy as np
import pandas
from pathlib import Path
import music21 as m21

sys.path.append(os.path.dirname(os.getcwd()))
from src.models.inference import single_piece_predict
from src.models.models import PKSpell
from src.models.process_score import process_score

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using", device)

torch.serialization.add_safe_globals([PKSpell])
torch.serialization.add_safe_globals([torch.nn.modules.rnn.GRU])
torch.serialization.add_safe_globals([torch.nn.modules.linear.Linear])
torch.serialization.add_safe_globals([torch.nn.modules.loss.CrossEntropyLoss])

model = torch.load(Path("../models/pkspell.pt"))

# process_score("tests/test_scores/bach_bwv867P_wrong.xml", "tests/test_scores/pkspelled_testscore.xml", model, device)

def extract1(directory_path, score_suffix):
    """return the first file with given suffix found in the given directory path"""
    files = os.listdir(directory_path)
    for file in files:    
        file_ext = os.path.splitext(file)[1]   # get the extension in the file name
        if (file_ext in score_suffix):
            return directory_path/file
    return # not found
    
def get_corpus(dataset_path, flat=True):
    """build a dictionary of XML scores with corresponding paths"""
    """flat: all XML files are contained in the given path or in subdirs"""
    # default score file name
    score_suffix = ['.musicxml', '.xml', '.mxml']
    if not os.path.exists(dataset_path):
        print('Error: ', dataset_path, 'not found')
        return
    # map: opus_name -> path
    dataset = dict()
    files = os.listdir(dataset_path)
    for file in files:    
        filepath = dataset_path/file
        # skip directories
        if not flat and os.path.isdir(filepath):
            target = extract1(filepath, score_suffix)
            if target is not None:
                dataset[file] = target
        # check the extension in the file name
        elif flat and (os.path.splitext(file)[1] in score_suffix):
            # map score name to file path
            dataset[os.path.splitext(file)[0]] = filepath
    # sort the list alphabetically
    dataset = dict(sorted(dataset.items()))
    return dataset
    
def extract_measure(m, notes, csflag=9):
	for e in m: 
		# case of single note
		if isinstance(e, m21.note.Note):
			notes.append(e)
		# case of chord symbol in jazz harmony (descendant of Chord)
		elif isinstance(e, m21.chord.Chord) and \
				(csflag != 0 or not isinstance(e, m21.harmony.ChordSymbol)):
			for cn in e:
				assert(isinstance(cn, m21.note.Note))
				notes.append(cn)

def extract_part(part, csflag=9):
	notes = []
	mes = part.getElementsByClass(m21.stream.Measure)
	for m in mes:			
		extract_measure(m, notes, csflag)
	return notes

def extract_score(score, csflag=9):
	"""extract the notes to be spelled from a score"""
	"""csflag=0: add the same notes as for PSE evaluation, ignoring Chord Symbols"""
	"""csflag=1: add the same notes as for PSE evaluation, spelling notes of Chord Symbols"""
	"""other csflag: PKspell standard option"""
	if csflag in [0, 1]: # analogous to PSE eval
		lp = score.getElementsByClass(m21.stream.Part)
		assert(len(lp) > 0)
		if (len(lp) > 1):
			print(len(lp), 'parts', 'spelling only the first')
		return extract_part(lp[0], csflag)		
	else: # PKspell defaults
		return score.flat.notes

def diff_list(notes, spelled):
	"""diff list computed under same condition as in PSE eval"""
	assert(len(notes) == len(spelled[0]))
	dl = []
	for i in range(len(notes)):
		skip_tie = (notes[i].tie is not None) and (notes[i].tie.type != 'start')
		if not skip_tie and (notes[i].pitches[0].step != m21.pitch.Pitch(spelled[0][i]).step):
			dl.append(i)
	return dl

def eval_score(score, csflag=9):
	notes = extract_score(score, csflag)
	# pitch class of each note (or chord symbol)
	pcs = [p.midi % 12 for n in notes for p in n.pitches]
	# duration of each note (or chord symbol)
	durs = [n.duration.quarterLength for n in notes for p in n.pitches]
	assert(len(pcs) == len(durs))
	ks_gt = score.getElementsByClass(m21.stream.Part)[0].getElementsByClass(m21.stream.Measure)[0].getElementsByClass([m21.key.Key, m21.key.KeySignature])[0].sharps
      # spell with PKspell
	spelled = single_piece_predict(pcs, durs, model, device)
	ks = spelled[1][0]
	assert(len(spelled[0]) == len(notes))
	#res = list(zip(notes, spelled[0]))
	#dl = [(res[i][0], res[i][1]) for i in range(len(res)) if res[i][0].pitches[0].step != m21.pitch.Pitch(res[i][1]).step]
	dl = diff_list(notes, spelled)
	return(ks_gt,  ks, len(pcs), len(dl))

def eval_opus(dataset, name, csflag=9):
	assert(len(name) > 0)
	if (dataset.get(name) == None):
		print(name, "not found in dataset")
		return
	file = dataset[name]
	score = m21.converter.parse(file.as_posix())
	return eval_score(score, csflag)

def eval_corpus(dataset, skip = [], csflag=9):
	names = sorted(list(dataset)) 
	table = []
	for name in names:
		if (name in skip):
			print('\n', name, 'SKIP\n', flush=True)
			continue
		print('\n evaluation of', name)
		(ks_gt,  ks, nn, err) = eval_opus(dataset, name, csflag)
		print('estimated KS:', ks, 'ground truth:', ks_gt)
		assert(nn > 0)
		pc = (nn - err) * 100 / nn
		print('diff:', err, 'on', nn, 'notes', "{:.2f}".format(pc), '%')
		if (ks_gt == ks):
			table.append([name, '',  ks, nn, err, pc])
		else:
			table.append([name, ks_gt,  ks, nn, err, pc])
	df = pandas.DataFrame(table)
	df.columns = ['name', 'KSgt', 'KSest', 'notes', 'err', 'success']
    # every KSestimated identical to corresp. KSgt becomes NaN
	df.loc[df['KSgt'] == '', 'KSgt'] = np.nan
	# line total
	df.at['total', 'notes'] = df['notes'].sum()
	df.at['total', 'err'] = df['err'].sum()
	df.at['total', 'KSgt'] = len(table)
	df.at['total', 'KSest'] = df['KSgt'].isna().sum() # correct extimations of KS
	# line success rate
	nbn = df.at['total', 'notes']
	assert(nbn > 0)
	df.at['rate', 'success'] = (nbn - df.at['total', 'err'])*100/nbn
	assert(len(table) > 0)
	df.at['rate', 'KSest'] = df.at['total', 'KSest']*100/len(table)
	#df.at['rate', 'KSest'] = df.at['rate', 'KSest'].map('{:,.2f}'.format) 
	#df.at['percent', 'KSest'].apply('{:,.2f}'.format) 
	df['success'] = df['success'].map('{:,.2f}'.format) 
	df = df.convert_dtypes()
	output = 'csflag1' if csflag == 1 else 'csflag0' if csflag == 0 else 'default'
	#df.fillna('').to_csv(output+'.csv', header=True, index=False)
	df.to_csv(output+'.csv', header=True, index=False)
	return df

def FRB():
	# flag: flat corpus
	skip = ['Freddie the freeloader']
	return (get_corpus(Path('../../../Datasets')/'FakeRealBook'/'leads', True), skip)

def omnibook():
	# flag: flat corpus
	return get_corpus(Path('../../../Datasets')/'CharlieParkerOmnibook'/'musescore', True)  

def FiloSax(transposed=False):
	if transposed: # flag: flat corpus
		return get_corpus(Path('../../../Datasets')/'FiloSax-xml'/'transpose_ms', True)  
	else:	
		return get_corpus(Path('../../../Datasets')/'FiloSax-xml', True)  

def FiloBass():		
	return get_corpus(Path('../../../Datasets')/'FiloBass-xml', True)  

Using cpu


In [ ]:
frb = FRB()

In [ ]:
eval_corpus(dataset=frb[0], skip=frb[1], csflag=0)


 evaluation of Afro blue
estimated KS: -4 ground truth: -4
diff: 0 on 75 notes 100.00 %

 evaluation of Afternoon in Paris
estimated KS: 0 ground truth: 0
diff: 0 on 110 notes 100.00 %

 evaluation of Airegin
estimated KS: -4 ground truth: -4
diff: 1 on 70 notes 98.57 %

 evaluation of Alice in wonderland
estimated KS: 0 ground truth: 0
diff: 0 on 85 notes 100.00 %

 evaluation of All blues
estimated KS: 1 ground truth: 0
diff: 1 on 37 notes 97.30 %

 evaluation of All in love is fair
estimated KS: -1 ground truth: -1
diff: 0 on 106 notes 100.00 %

 evaluation of All my tomorrows
estimated KS: 1 ground truth: 1
diff: 5 on 130 notes 96.15 %

 evaluation of All of me
estimated KS: 0 ground truth: 0
diff: 0 on 85 notes 100.00 %

 evaluation of All of you
estimated KS: -3 ground truth: -3
diff: 2 on 79 notes 97.47 %

 evaluation of All the things you are
estimated KS: -4 ground truth: -4
diff: 2 on 90 notes 97.78 %

 evaluation of Alone together
estimated KS: -1 ground truth: -1
diff: 1 o

,name,KSgt,KSest,notes,err,success
0,Afro blue,<NA>,-4.0,75,0,100.00
1,Afternoon in Paris,<NA>,0.0,110,0,100.00
2,Airegin,<NA>,-4.0,70,1,98.57
3,Alice in wonderland,<NA>,0.0,85,0,100.00
4,All blues,0,1.0,37,1,97.30
...,...,...,...,...,...,...
198,Lulu's back in town,<NA>,-3.0,100,0,100.00
199,Misty,<NA>,-3.0,126,1,99.21
200,The fields we know,0,5.0,64,16,75.00
total,<NA>,201,131.0,20399,671,nan


In [ ]:
eval_corpus(dataset=frb[0], skip=frb[1], csflag=1)


 evaluation of Afro blue
estimated KS: -4 ground truth: -4
diff: 0 on 215 notes 100.00 %

 evaluation of Afternoon in Paris
estimated KS: 0 ground truth: 0
diff: 0 on 246 notes 100.00 %

 evaluation of Airegin
estimated KS: -4 ground truth: -4
diff: 11 on 218 notes 94.95 %

 evaluation of Alice in wonderland
estimated KS: 0 ground truth: 0
diff: 5 on 302 notes 98.34 %

 evaluation of All blues
estimated KS: 1 ground truth: 0
diff: 3 on 69 notes 95.65 %

 evaluation of All in love is fair
estimated KS: -1 ground truth: -1
diff: 2 on 282 notes 99.29 %

 evaluation of All my tomorrows
estimated KS: 1 ground truth: 1
diff: 16 on 319 notes 94.98 %

 evaluation of All of me
estimated KS: 0 ground truth: 0
diff: 6 on 176 notes 96.59 %

 evaluation of All of you
estimated KS: -3 ground truth: -3
diff: 3 on 228 notes 98.68 %

 evaluation of All the things you are
estimated KS: -3 ground truth: -4
diff: 3 on 226 notes 98.67 %

 evaluation of Alone together
estimated KS: -1 ground truth: -1
diff

,name,KSgt,KSest,notes,err,success
0,Afro blue,<NA>,-4.0,215,0,100.00
1,Afternoon in Paris,<NA>,0.0,246,0,100.00
2,Airegin,<NA>,-4.0,218,11,94.95
3,Alice in wonderland,<NA>,0.0,302,5,98.34
4,All blues,0,1.0,69,3,95.65
...,...,...,...,...,...,...
198,Lulu's back in town,<NA>,-3.0,301,9,97.01
199,Misty,-3,-4.0,288,2,99.31
200,The fields we know,0,3.0,151,58,61.59
total,<NA>,201,140.0,52116,2216,nan
